## Universidad del Valle de Guatemala  
## Departamento de Ciencias de la Computación  
## Inteligencia Artificial - sección 10  

### Hoja de Trabajo 2

#### Nadissa Vela - 23764  
#### Cristian Túnchez - 231359

---


# Task 2 - Implementación Frozen Lake en Python

El agente debe cruzar un lago congelado representado por un grid de 4x4 donde:
- **S (Start):** Posición inicial
- **F (Frozen):** Camino seguro
- **H (Hole):** Agujero - Termina el juego con recompensa 0
- **G (Goal):** Meta - Recompensa +1

**Dinámica Estocástica:**  
El hielo es resbaladizo. Al tomar una acción, hay:
- 1/3 de probabilidad de moverse en la dirección deseada
- 1/3 de probabilidad de moverse a cada dirección perpendicular

Ejemplo: Si intenta ir Norte, puede terminar en Norte, Este u Oeste con igual probabilidad.

In [ ]:
# Importación de bibliotecas permitidas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap

---
## Task 2.1 - Modelado del MDP

Implementaremos una clase `FrozenLakeMDP` que modele completamente el problema como un Proceso de Decisión de Markov.

### Componentes del MDP

Según la teoría, un MDP se define por:
1. **Estados (S):** Conjunto de todos los estados posibles
2. **Estado inicial (s_start):** Estado donde comienza el agente
3. **Acciones (Actions(s)):** Acciones disponibles en cada estado
4. **Función de transición (T(s, a, s')):** Probabilidad de llegar a s' al realizar acción a en estado s
5. **Función de recompensa (Reward(s, a, s')):** Recompensa recibida por la transición
6. **Estados finales (IsEnd(s)):** Indica si un estado es terminal
7. **Factor de descuento (γ):** Importancia de recompensas futuras (usaremos γ=0.9)

In [ ]:
class FrozenLakeMDP:
    """
    Implementación del problema Frozen Lake como un Markov Decision Process.
    
    Grid 4x4 con numeración de estados:
    [ 0  1  2  3]
    [ 4  5  6  7]
    [ 8  9 10 11]
    [12 13 14 15]
    
    Mapa del lago:
    S F F F
    F H F H
    F F F H
    H F F G
    
    Donde:
    S = Start (estado 0)
    F = Frozen (camino seguro)
    H = Hole (agujero, estado terminal)
    G = Goal (meta, estado terminal)
    """
    
    def __init__(self, gamma=0.9):
        """
        Inicializa el MDP del Frozen Lake.
        
        Args:
            gamma (float): Factor de descuento γ ∈ [0,1] para recompensas futuras
        """
        # Factor de descuento γ
        self.gamma = gamma
        
        # Dimensiones del grid
        self.rows = 4
        self.cols = 4
        self.n_states = self.rows * self.cols  # 16 estados (0-15)
        
        # Definición del mapa del lago (usando el layout estándar de Frozen Lake)
        # S: Start, F: Frozen, H: Hole, G: Goal
        self.lake_map = [
            ['S', 'F', 'F', 'F'],
            ['F', 'H', 'F', 'H'],
            ['F', 'F', 'F', 'H'],
            ['H', 'F', 'F', 'G']
        ]
        
        # Estado inicial
        self.start_state = 0  # Esquina superior izquierda
        
        # Estados terminales (Holes y Goal)
        # Holes: 5, 7, 11, 12
        # Goal: 15
        self.hole_states = {5, 7, 11, 12}
        self.goal_state = 15
        self.terminal_states = self.hole_states | {self.goal_state}
        
        # Definición de acciones: 0=Norte, 1=Sur, 2=Este, 3=Oeste
        self.actions = [0, 1, 2, 3]
        self.action_names = ['Norte', 'Sur', 'Este', 'Oeste']
        self.action_symbols = ['↑', '↓', '→', '←']
        
        # Movimientos correspondientes a cada acción
        # (delta_row, delta_col)
        self.action_deltas = {
            0: (-1, 0),  # Norte: arriba
            1: (1, 0),   # Sur: abajo
            2: (0, 1),   # Este: derecha
            3: (0, -1)   # Oeste: izquierda
        }
        
        # Precalcular las direcciones perpendiculares para cada acción
        # Para modelar la estocasticidad del hielo resbaladizo
        self.perpendicular_actions = {
            0: [2, 3],  # Norte -> perpendiculares son Este y Oeste
            1: [2, 3],  # Sur -> perpendiculares son Este y Oeste
            2: [0, 1],  # Este -> perpendiculares son Norte y Sur
            3: [0, 1]   # Oeste -> perpendiculares son Norte y Sur
        }
    
    def state_to_position(self, state):
        """
        Convierte un número de estado (0-15) a posición (row, col) en el grid.
        
        Args:
            state (int): Número de estado
            
        Returns:
            tuple: (row, col)
        """
        row = state // self.cols
        col = state % self.cols
        return row, col
    
    def position_to_state(self, row, col):
        """
        Convierte una posición (row, col) a número de estado.
        
        Args:
            row (int): Fila
            col (int): Columna
            
        Returns:
            int: Número de estado
        """
        return row * self.cols + col
    
    def is_valid_position(self, row, col):
        """
        Verifica si una posición está dentro del grid.
        
        Args:
            row (int): Fila
            col (int): Columna
            
        Returns:
            bool: True si la posición es válida
        """
        return 0 <= row < self.rows and 0 <= col < self.cols
    
    def is_terminal(self, state):
        """
        Verifica si un estado es terminal (IsEnd(s)).
        
        Estados terminales: Holes (5, 7, 11, 12) y Goal (15)
        
        Args:
            state (int): Número de estado
            
        Returns:
            bool: True si el estado es terminal
        """
        return state in self.terminal_states
    
    def get_next_state(self, state, action):
        """
        Calcula el estado resultante de tomar una acción desde un estado.
        No considera estocasticidad, solo calcula el resultado determinístico.
        
        Si el movimiento sale del grid, el agente permanece en el mismo estado.
        
        Args:
            state (int): Estado actual
            action (int): Acción a realizar (0=Norte, 1=Sur, 2=Este, 3=Oeste)
            
        Returns:
            int: Estado siguiente
        """
        # Si ya estamos en un estado terminal, nos quedamos ahí
        if self.is_terminal(state):
            return state
        
        # Calcular nueva posición
        row, col = self.state_to_position(state)
        delta_row, delta_col = self.action_deltas[action]
        new_row = row + delta_row
        new_col = col + delta_col
        
        # Verificar si la nueva posición es válida
        if self.is_valid_position(new_row, new_col):
            return self.position_to_state(new_row, new_col)
        else:
            # Si el movimiento sale del grid, permanecemos en el estado actual
            return state
    
    def get_transition_prob(self, state, action, next_state):
        """
        Calcula la probabilidad de transición T(s, a, s').
        
        Implementa la dinámica estocástica del hielo resbaladizo:
        - Probabilidad 1/3 de moverse en la dirección deseada
        - Probabilidad 1/3 de moverse en cada dirección perpendicular
        
        Fórmula del MDP:
        T(s, a, s') = P(s_{t+1} = s' | s_t = s, a_t = a)
        
        Args:
            state (int): Estado actual s
            action (int): Acción a
            next_state (int): Estado siguiente s'
            
        Returns:
            float: Probabilidad de transición T(s, a, s')
        """
        # Si estamos en un estado terminal, la probabilidad de quedarse es 1
        if self.is_terminal(state):
            return 1.0 if next_state == state else 0.0
        
        # Obtener los estados resultantes de la acción deseada y las perpendiculares
        intended_state = self.get_next_state(state, action)
        perp_actions = self.perpendicular_actions[action]
        perp_state1 = self.get_next_state(state, perp_actions[0])
        perp_state2 = self.get_next_state(state, perp_actions[1])
        
        # Contar cuántas veces aparece next_state en los posibles resultados
        # Cada aparición contribuye con probabilidad 1/3
        possible_states = [intended_state, perp_state1, perp_state2]
        count = possible_states.count(next_state)
        
        return count / 3.0
    
    def get_reward(self, state, action, next_state):
        """
        Función de recompensa Reward(s, a, s').
        
        Definición de recompensas:
        - Llegar al Goal (estado 15): +1
        - Llegar a un Hole: 0 (el juego termina sin recompensa)
        - Cualquier otra transición: 0
        
        Args:
            state (int): Estado actual s
            action (int): Acción a
            next_state (int): Estado siguiente s'
            
        Returns:
            float: Recompensa R(s, a, s')
        """
        # Recompensa de +1 solo al alcanzar el Goal
        if next_state == self.goal_state:
            return 1.0
        else:
            return 0.0
    
    def get_possible_next_states(self, state, action):
        """
        Obtiene todos los posibles estados siguientes al tomar una acción,
        junto con sus probabilidades de transición.
        
        Útil para la implementación eficiente de Value Iteration.
        
        Args:
            state (int): Estado actual
            action (int): Acción
            
        Returns:
            list: Lista de tuplas (next_state, probability, reward)
        """
        if self.is_terminal(state):
            return [(state, 1.0, 0.0)]
        
        # Calcular los tres posibles resultados
        intended_state = self.get_next_state(state, action)
        perp_actions = self.perpendicular_actions[action]
        perp_state1 = self.get_next_state(state, perp_actions[0])
        perp_state2 = self.get_next_state(state, perp_actions[1])
        
        # Agrupar estados idénticos y sumar sus probabilidades
        state_probs = {}
        for next_s in [intended_state, perp_state1, perp_state2]:
            if next_s in state_probs:
                state_probs[next_s] += 1/3.0
            else:
                state_probs[next_s] = 1/3.0
        
        # Crear lista de resultados con recompensas
        results = []
        for next_s, prob in state_probs.items():
            reward = self.get_reward(state, action, next_s)
            results.append((next_s, prob, reward))
        
        return results
    
    def visualize_grid(self):
        """
        Visualiza el mapa del Frozen Lake.
        """
        print("\nMapa del Frozen Lake (4x4):")
        print("="*30)
        for i, row in enumerate(self.lake_map):
            print(f"  {' '.join(row)}    (estados {i*4} a {i*4+3})")
        print("="*30)
        print("S = Start (inicio)")
        print("F = Frozen (camino seguro)")
        print("H = Hole (agujero - terminal)")
        print("G = Goal (meta - terminal, recompensa +1)")
        print(f"\nEstados terminales: {sorted(self.terminal_states)}")
        print(f"Factor de descuento γ = {self.gamma}")

### Prueba del Modelado del MDP

Verificamos que la función de transición T(s, a, s') y la función de recompensa R(s, a, s') estén correctamente implementadas.

In [ ]:
# Crear instancia del MDP
mdp = FrozenLakeMDP(gamma=0.9)

# Visualizar el mapa
mdp.visualize_grid()

In [ ]:
# Ejemplo de función de transición T(s, a, s')
print("\nEjemplo de Función de Transición T(s, a, s')")
print("="*50)

# Desde el estado inicial (0), intentamos ir al Sur (acción 1)
state = 0
action = 1  # Sur

print(f"\nDesde estado {state}, acción: {mdp.action_names[action]} ({mdp.action_symbols[action]})")
print("\nDinámica estocástica del hielo resbaladizo:")
print(f"- Dirección deseada (Sur): estado {mdp.get_next_state(state, 1)}")
print(f"- Dirección perpendicular 1 (Este): estado {mdp.get_next_state(state, 2)}")
print(f"- Dirección perpendicular 2 (Oeste): estado {mdp.get_next_state(state, 3)}")

print("\nProbabilidades de transición T(s, a, s'):")
for s_prime in range(mdp.n_states):
    prob = mdp.get_transition_prob(state, action, s_prime)
    if prob > 0:
        print(f"  T({state}, {mdp.action_names[action]}, {s_prime}) = {prob:.3f}")

# Verificar que las probabilidades suman 1
total_prob = sum(mdp.get_transition_prob(state, action, s_prime) 
                 for s_prime in range(mdp.n_states))
print(f"\nSuma de probabilidades: {total_prob:.3f} (debe ser 1.0)")

In [ ]:
# Ejemplo de función de recompensa R(s, a, s')
print("\nEjemplo de Función de Recompensa R(s, a, s')")
print("="*50)

# Transición al Goal
print(f"\nLlegar al Goal (estado {mdp.goal_state}):")
print(f"  R(14, Este, 15) = {mdp.get_reward(14, 2, 15)}")

# Transición a un Hole
print(f"\nLlegar a un Hole (estado 5):")
print(f"  R(1, Sur, 5) = {mdp.get_reward(1, 1, 5)}")

# Transición normal
print(f"\nTransición normal (sin llegar a terminal):")
print(f"  R(0, Este, 1) = {mdp.get_reward(0, 2, 1)}")